| Elemento    | Definição |
| -------- | ------- |
| **Estado Inicial**  |   Matriz do jogo da velha vazia   |
| **Estado Objetivo** | Formar uma sequência de três "X" na horizontal, vertical ou diagonal |
| **Função Sucessor** | Jogadores escolhem um elemento não marcado para marcarem com seu respectivo síbolo ("X" ou "O") |
| **Custo de Caminho** | Ganhou o jogo = 1, Perdeu o Jogo = -1, Empatou = 0 |

**Resumo:**
O jogo se incia em uma matriz 3x3 e o objetivo do jogador é formar uma sequência de três símbolos iguais na horizontal, vertical ou diagonal antes que seu oponente possa fazer o mesmo. Cada jogador pode colocar um símbolo por vês da matriz. O jogo termina quando um dos jogadores completar a sequência descrita anteriormente ou a matriz ser completamente preenchida sem um resultado final.

Será usado um algoritmo minimax, sendo que o jogo possui estado deterministico.


In [28]:
import math

def print_board(state):
    for row in state:
        print("|".join(cell if cell != '' and cell != ' ' else ' ' for cell in row))
    print()

def trocar_jogador(player):
    return 'O' if player == 'X' else 'X'

def checar_vencedor(lista):
    for sequencia in lista:
        conjunto_sequencia = set(sequencia)
        if len(conjunto_sequencia) == 1:
            vencedor = conjunto_sequencia.pop()
            if vencedor in ('X', 'O'):
                return vencedor
    return None

def determinar_termino(linhas):
    colunas = list(zip(*linhas))
    diagonal_1 = [linha[i] for i, linha in enumerate(linhas)]
    diagonal_2 = [linha[2-i] for i, linha in enumerate(linhas)]
    direcoes = linhas + list(colunas) + [diagonal_1, diagonal_2]
    vencedor = checar_vencedor(direcoes)
    if vencedor in ('X','O'):
        return vencedor
    
    elementos = [elemento for linha in linhas for elemento in linha]
    if ' ' not in elementos:
        return ' '
    
    return None

def determinar_pontuacao(estado):
    termino = determinar_termino(estado['matriz'])
    if termino == 'X':
        estado['termino'] = True
        estado['pontos'] = 1
    elif termino == 'O':
        estado['termino'] = True
        estado['pontos'] = -1
    elif termino == ' ':
        estado['termino'] = True
        estado['pontos'] = 0
    else:
        estado['termino'] = False
        estado['pontos'] = 0
    return estado

def sucessores(estado, jogador):
    sucessores = []
    for i in range(3):
        for j in range(3):
            if estado['matriz'][i][j] == ' ':
                new_matriz = [row.copy() for row in estado['matriz']]
                new_matriz[i][j] = jogador
                novo_estado = {
                    'prox': trocar_jogador(jogador),
                    'matriz': new_matriz,
                    'pontos': 0,
                    'termino': False
                }
                determinar_pontuacao(novo_estado)
                sucessores.append(novo_estado)
    return sucessores

def max_value(estado, alfa, beta):
    if estado["termino"]:
        return estado['pontos'], estado

    v = float('-inf')
    melhor_estado = None
    for sucessor in sucessores(estado, 'X'):
        valor_sucessor, _ = valor(sucessor, alfa, beta)
        if valor_sucessor > v:
            v = valor_sucessor
            melhor_estado = sucessor
        if v >= beta:
            return v, melhor_estado
        alfa = max(alfa, v)
    return v, melhor_estado

def min_value(estado, alfa, beta):
    if estado["termino"]:
        return estado['pontos'], estado

    v = float('inf')
    melhor_estado = None
    for sucessor in sucessores(estado, 'O'):
        valor_sucessor, _ = valor(sucessor, alfa, beta)
        if valor_sucessor < v:
            v = valor_sucessor
            melhor_estado = sucessor
        if v <= alfa:
            return v, melhor_estado
        beta = min(beta, v)
    return v, melhor_estado

def valor(estado, alfa, beta):
    if estado['termino']: 
        return estado['pontos'], estado
    if estado['prox'] == 'X': 
        return max_value(estado, alfa, beta)
    if estado['prox'] == 'O':
        return min_value(estado, alfa, beta)

def iniciar_jogo():
    matriz = [
        [' ', ' ', ' '],
        [' ', ' ', ' '],
        [' ', ' ', ' ']
    ]
    estado = {
        'prox': 'X',
        'matriz': matriz,
        'pontos': 0,
        'termino': False
    }

    print("Estado inicial do tabuleiro:")
    print_board(estado['matriz'])

    while not estado['termino']:
        if estado['prox'] == 'X':
            _, novo_estado = max_value(estado, -math.inf, math.inf)
        else:
            _, novo_estado = min_value(estado, -math.inf, math.inf)
        estado = novo_estado
        print(f"Após jogada do {trocar_jogador(estado['prox'])}:")
        print_board(estado['matriz'])

    vencedor = determinar_termino(estado['matriz'])
    if vencedor == 'X' or vencedor == 'O':
        print(f"Jogador {vencedor} ganhou!")
    else:
        print("Empate!")

iniciar_jogo()


Estado inicial do tabuleiro:
 | | 
 | | 
 | | 

Após jogada do X:
X| | 
 | | 
 | | 

Após jogada do O:
X| | 
 |O| 
 | | 

Após jogada do X:
X|X| 
 |O| 
 | | 

Após jogada do O:
X|X|O
 |O| 
 | | 

Após jogada do X:
X|X|O
 |O| 
X| | 

Após jogada do O:
X|X|O
O|O| 
X| | 

Após jogada do X:
X|X|O
O|O|X
X| | 

Após jogada do O:
X|X|O
O|O|X
X|O| 

Após jogada do X:
X|X|O
O|O|X
X|O|X

Empate!
